In [1]:
from neo4j import GraphDatabase
from py2neo import Graph
import pandas as pd

In [2]:
import requests
import json

url = "https://robokop-automat.apps.renci.org/robokopkg/cypher"

payload = json.dumps({"query": """MATCH (c:`biolink:ChemicalEntity`)-[r0:`biolink:binds`|`biolink:directly_physically_interacts_with`]-(g:`biolink:Gene`)-[r1]-(d:`biolink:Disease`),(c)-[r2:`biolink:treats`]-(d) WHERE (properties(c)["CHEBI_ROLE_pharmaceutical"]) IS NOT NULL AND properties(r2)["primary_knowledge_source"]="infores:drugcentral" RETURN DISTINCT c.name,c.id,d.name,d.id,g.name,g.id"""
})
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json'
}

response = requests.post(url, headers=headers, data=payload)
print(response)
response_json = response.json()


KeyboardInterrupt: 

In [135]:
values = response_json
num_results = len(values['results'][0]['data'])
triples_list = []
for i in range(num_results):
    data = values['results'][0]['data'][i]['row']
    triples_list.append(data)

for i in range(len(triples_list)):
    triples_list[i][4] = triples_list[i][4].lower()
    triples_list[i][0] = triples_list[i][0].lower()
    triples_list[i][2] = triples_list[i][2].lower()

print(triples_list[:4])

[['selumetinib', 'CHEBI:90227', 'neurofibromatosis 1', 'DOID:0111253', 'smc1a', 'NCBIGene:8243'], ['selumetinib', 'CHEBI:90227', 'neurofibromatosis 1', 'DOID:0111253', 'smc2', 'NCBIGene:10592'], ['selumetinib', 'CHEBI:90227', 'neurofibromatosis 1', 'DOID:0111253', 'map2k1', 'NCBIGene:5604'], ['selumetinib', 'CHEBI:90227', 'neurofibromatosis 1', 'DOID:0111253', 'map2k2', 'NCBIGene:5605']]


In [136]:
print(len(triples_list))

8063


In [137]:
import csv
import os
robokop_data_dir = os.path.join(os.path.dirname(os.getcwd()), 'ROBOKOP Data')

df = pd.DataFrame(data = triples_list,columns = ['drug_name','drug_id', 'disease_name','disease_id','gene_name','gene_id'])
df.to_csv(os.path.join(robokop_data_dir, 'ROBOKOP Processed Triples.csv'),quoting=csv.QUOTE_ALL)